In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
from keras import Sequential
from keras.layers import LSTM

2023-04-16 10:35:06.238587: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


LSTM Model Implementation:

In [2]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [3]:
train = train_df.copy()[:int(len(train_df))]
train = train.set_index("dates")
train = np.array(train)

In [4]:
trainX = []
trainY = []
n_past = 4
n_future = 1
for i in range(n_past, len(train)):
    trainX.append(train[i - n_past:i, 0:14])
    trainY.append(train[i: i + 1, -3:])

trainX, trainY = np.array(trainX), np.array(trainY)
trainY = trainY.reshape((len(trainX), 3))

Selection of Appropriate hyperparameters:

In [5]:
model = Sequential()
model.add(LSTM(180, input_shape=(n_past,14), return_sequences=True))
model.add(LSTM(90, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(48, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='linear'))

2023-04-16 10:35:11.889822: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-16 10:35:11.892153: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-16 10:35:11.894351: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [6]:
model.compile(optimizer='adam', loss='mse')

In [7]:
model.fit(trainX, trainY, epochs=2)

Epoch 1/2


2023-04-16 10:35:13.100919: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-16 10:35:13.105681: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-16 10:35:13.109924: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

5056/5056 [==============================] - 81s 15ms/step - loss: 2168.8264
Epoch 2/2
5056/5056 [==============================] - 81s 16ms/step - loss: 309.0813


In [8]:
test = test_df.copy()[:int(len(test_df))]
test = test.set_index("dates")
test = np.array(test)
testX = []
n_future = 1
for i in range(n_past, len(test_df)):
    testX.append(train[i - n_past:i, 0:14])

testX = np.array(testX)

In [9]:
y_pred = model.predict(testX)
y_pred = np.sum(y_pred, axis=1)

2023-04-16 10:37:54.988583: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-16 10:37:54.990174: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-16 10:37:54.992391: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

938/938 [==============================] - 5s 4ms/step


In [10]:
for i in range(n_past):
    y_pred = np.append(y_pred, y_pred.mean())

In [11]:
out = pd.DataFrame()
out['id'] = [i for i in range(len(y_pred))]
out['predicted'] = y_pred
out.to_csv("sub10.csv", index=False)

In [12]:
out

,id,predicted
0,0,1430.248291
1,1,1430.248291
2,2,1430.248291
3,3,1430.248291
4,4,1430.248291
...,...,...
29995,29995,1430.248291
29996,29996,1430.221191
29997,29997,1430.221313
29998,29998,1430.221191


In [43]:
import datetime

date_str = '2142-11-27'

date_obj = datetime.datetime.strptime(date_str, '%Y-%m-%d')

dates = []
for i in range (5):
    date_obj+=datetime.timedelta(days=1)
    dates.append(date_obj)

df = pd.DataFrame({'Date':dates})
#df = df.reset_index()
df.head()


,Date
0,2142-11-28
1,2142-11-29
2,2142-11-30
3,2142-12-01
4,2142-12-02


In [44]:
out.head()

,id,predicted
0,0,1430.248291
1,1,1430.248291
2,2,1430.248291
3,3,1430.248291
4,4,1430.248291


In [45]:
df_main = pd.concat([df, out], axis=1)
df_main.tail()

,Date,id,predicted
29995,NaT,29995,1430.248291
29996,NaT,29996,1430.221191
29997,NaT,29997,1430.221313
29998,NaT,29998,1430.221191
29999,NaT,29999,1430.221313


In [32]:
df_main = df_main.drop('id', axis = 1)
df_main.head()

,Date,predicted
0,2142-11-28,1430.248291
1,2142-11-29,1430.248291
2,2142-11-30,1430.248291
3,2142-12-01,1430.248291
4,2142-12-02,1430.248291


In [42]:
import plotly.express as px

fig = px.line(df_main, x='Date', y='predicted')
fig.update_layout(title="final prediction")
fig.show()